# Claycode SIGGRAPH Evaluation

In [85]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


df = pd.read_csv('results.csv')

df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))
df['square_area_perc'] = df['square_dimension_perc']**2

df = df[['successful', 'square_area_perc', 'type']]

# Turn the square side dimension into an area percentage
grouped = df.groupby(['square_area_perc', 'type'])

# Aggregate the 'successful' column
df = grouped['successful'].agg(['sum', 'count']).reset_index()

# Calculate success rate
df['success_rate'] = df['sum'] / df['count']

In [86]:
def hex_to_rgb(hex_color):
    """Convert a hexadecimal color code to an RGB tuple."""
    hex_color = hex_color.lstrip('#')  # Remove the '#' if it exists
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))


# Define all pairs to display
trace_types = [
    'qrart', 'qr', 'claycodeart', 'claycode','claycode-fragment', 'barcode'
]

name_mapping = {
    'claycode': 'Claycode (R=1)',
    'claycode-fragment': 'Claycode (R=2)',
    'qr': 'QR',
    'qrart': 'Styl. QR',
    'claycodeart': 'Styl. Claycode (R=2)',
    'barcode': 'Barcode'
}

##################
# Colors
##################

claycode_color_main = '#156b59'  
claycode_color_variant2 = '#298c73'  
claycode_color_variant1 = '#26a78a' 
qr_color_main = '#d95f02'  # Main shade for qr
qr_color_variant = '#f5861a'  # orange for variant
barcode_color = '#1160b8' 

color_scheme = {
    'claycode': claycode_color_main, 
    'claycodeart': claycode_color_variant1, 
    'claycode-fragment': claycode_color_variant2, 
    'qr': qr_color_main, 
    'qrart': qr_color_variant, 
    'barcode': barcode_color,
    'claycode2': claycode_color_main, 
    'claycodeart2': claycode_color_variant1, 
    'claycode2-fragment': claycode_color_variant2, 
    'qr2': qr_color_main, 
    'qrart2': qr_color_variant, 
    'barcode2': barcode_color
}

##################
# Marker styles
##################

marker_qr = 'square'
marker_qrart = 'diamond'
marker_claycode = 'circle'
marker_claycodeart = 'bowtie'
marker_claycode_fragment = 'diamond-tall'
marker_barcode = 'x'
marker_map = {
    'qr': marker_qr,
    'qrart': marker_qrart,
    'claycode': marker_claycode,
    'claycodeart': marker_claycodeart,
    'barcode': marker_barcode,
    'claycode-fragment': marker_claycode_fragment,
}

MULT = 1.1
SIZE_QR = 9
SIZE_QR_ART = 11
SIZE_CLAYCODE = 8
SIZE_CLAYCODE_ART = 12
SIZE_CLAYCODE_FRAGMENT = 10
SIZE_BARCODE = 9
marker_size_map = {
    'claycode': SIZE_CLAYCODE,
    'claycodeart': SIZE_CLAYCODE_ART,
    'claycode-fragment': SIZE_CLAYCODE_FRAGMENT,
    'qr': SIZE_QR,
    'qrart': SIZE_QR_ART,
    'barcode': SIZE_BARCODE,
}
marker_size_map = { key: value*MULT for key,value in marker_size_map.items() }


# Plot setup
fig = go.Figure()

for trace_type in trace_types:
    trace = df[df['type'] == trace_type]

    fig.add_trace(go.Scatter(
        x=trace['square_area_perc'],
        y=trace['success_rate'],
        mode='lines+markers',
        name=name_mapping.get(trace_type, trace_type),
        line=dict(
            color=color_scheme.get(trace_type, '#000000'),
        ),
        marker=dict(
            symbol=marker_map.get(trace_type, 'circle'),
            size=marker_size_map.get(trace_type, 0)
        )
    ))

FIG_MULT = 0.8
fig.update_layout(
    xaxis_title="Area Occluded (%)",
    height=410*FIG_MULT,
    width=615*FIG_MULT,
    showlegend=False,
    margin=dict(t=10, b=10, l=10, r=7)
)

fig.update_traces(opacity=1, line=dict(width=2.3))
fig.update_xaxes(range=[0.001, 0.255], tickformat='.0%', tickvals=[.01, .04, .09, .16, .25])
fig.update_yaxes(range=[-.035, 1.05], tickformat='.0%')

# Export to PDF
fig.write_image("occlusion.pdf", format="pdf", engine="kaleido")
fig.show()
